### CONSTANTS Variables

In [1]:
# CONSTANTS
DRIVER_PATH = "/home/saleh/Documents/INetwork Project/geckodriver"
SOURCE_URL = "https://www.udacity.com/courses/all" 
UDACITY_URL = 'https://www.udacity.com'
UDEMY_URL = 'https://www.udemy.com'
CLIENT_ID = 'jqRzWZZfdvjdtQpzEDmjGWVmhLMpTWHv5uN27li3'
CLIENT_SECRET = 'b8MjyR6TUXzCcXnFDIVrCpHbMpDAUrJW40xqV2zvBoagL6aAWZ9a0JAMg54TyYaOxZ9ksEwt2JpJmt98En5pszOnrHxrPrZlbg3pNCCcTb9SkmBwcCpnoWYddQjOOSPq'
list_of_required_keys = ['title', 'url', 'description', 'syllabus',\
                         'skills', 'ratings_count', 'prerequisites', 'duration',\
                         'category', 'level', 'schoolName', 'instructors', 'enrolled_students_count',\
                         'avg_rating', 'num_reviews', 'price', 'source']

### Import Libraries

In [2]:
# Import Libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from pyudemy import Udemy
import requests
import time
import re
import json 
from pymongo import MongoClient

### Setup Web Driver 

In [3]:
# Setup Firefox dirver to prevent pop-up
Firefox_option = Options()
Firefox_option.add_argument('--headless')
Firefox_option.add_argument('--no-sandbox')
Firefox_option.add_argument('--disable-dev-shm-usage')
driver = webdriver.Firefox(executable_path = DRIVER_PATH, options = Firefox_option)

### Udacity courses scraping

In [4]:
# Get html code from source page 
driver.get(SOURCE_URL)
# sleep time to scrape all info from page
time.sleep(5)

In [5]:
# Create BeautifulSoup object to scrape information from source page
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
# Scrape all information needed from the page of alll courses
courses_title = list(map( lambda x : x.text , soup.find_all('h2' , {"class" : "card_title__35G97"})))
courses_urls = list(map(lambda x : UDACITY_URL + x['href'] , soup.find_all('a' , {"class" : "card_container__25DrK"}) ))
courses_duration = list(map(lambda x : x.text , soup.find_all("span" , {"class" : "card_duration__1hWII"})))
description = list(map(lambda x : x.text , soup.find_all("p" , {"class" : "card_summary__1HlQ7"})))

skills = list(map( lambda x : x.text , soup.select("div[class='card_details__3VdjA'] > p:nth-of-type(1)")))
prerequisites = list(map( lambda x : x.text , soup.select("div[class='card_details__3VdjA'] > p:nth-of-type(2)")))

levels = list(map(lambda x : x.text , soup.select("div[class='card_stats__AV1N0'] > span:nth-of-type(1)")))
categories = list(map(lambda x : x.text , soup.select("div[class='card_flagContainer__2Jiia'] > div")))

In [7]:
# drop courses without prerequisites and skills
drop_courses = ['SQL for Data Analysis', 'Secure and Private AI', 'Intro to TensorFlow for Deep Learning',
                'What is Programming?', 'iOS Design Patterns', 'Passwordless Login Solutions for Android',
                'Configuring Linux Web Servers', 'JavaScript and the DOM', 'WeChat Mini Program Development',
                'Cyber-Physical Systems Design & Analysis', 'Introduction to Graduate Algorithms',
                'Introduction to Python Programming', 'Computability, Complexity & Algorithms']

In [8]:
# drop courses without prerequisites and skills from other lists related to courses
for drop_course in drop_courses:
    index = courses_title.index(drop_course)
    
    del(courses_title[index])
    del(courses_urls[index])
    del(courses_duration[index])
    del(description[index])
    del(levels[index])
    del(categories[index])

In [9]:
# add empty string to prerequisites to be the same number of courses
# prerequisites of free courses is empty
for i in range(len(courses_title)-len(prerequisites)):
    prerequisites.append('')

### Create List of Dictionay of courses scraped from Udacity

In [10]:
def map_new_keys_for_udacity(title, url, duration, description, skill, prerequisites, level, category):
    course_dict = {}
    
    def scraping_udacity_course(url, prerequisites):
        # This function to get missed information from main page
        # Get html code from source page 
        driver.get(url)
        time.sleep(5)
        
        # category is nanodegree
        if category == 'nanodegree':
            syllabus = 'Not Available'
            enrolled_students_count = 'Not Available'
            ret_prerequisites = prerequisites
            
            # exception handeling if the scraped features not found 
            try:
                avg_rating_temp = driver.find_elements_by_xpath("/html/body/div[1]/div/div/div[2]/section[9]/section/div/header/div/div[1]")
                avg_rating = avg_rating_temp[0].text
            except:
                avg_rating = 'Not Available'

            try:
                ratings_count_temp = driver.find_elements_by_xpath("/html/body/div[1]/div/div/div[2]/section[9]/section/div/header/div/span")
                ratings_count = ratings_count_temp[0].text[1:-1]
            except:
                ratings_count = 'Not Available'

            num_reviews = ratings_count

            instructors_temp = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div[2]/section[8]/div/div/div/div[1]/a/h5')

            instructors = []
            for name in instructors_temp:
                instructors.append(name.text)
            try:
                p = driver.find_elements_by_xpath("/html/body/div[1]/div/div/div[2]/section[10]/div/div/ul[1]/li[2]/figure/article/div[2]/p/ins")
                price =  p[0].text
            except:
                # price is none if price not scraped or not found
                price = 'None'

        # category is free      
        else:
            syllabus = 'Not Available'
            enrolled_students_count = 'Not Available'
            avg_rating = 'Not Available'
            ratings_count = 'Not Available'
            num_reviews = ratings_count
            
            ret_prerequisites_temp = driver.find_elements_by_xpath("/html/body/div[1]/div/div/div[2]/section[7]/section/div")
            
            ret_prerequisites = []
            for pre in ret_prerequisites_temp:
                ret_prerequisites.append(pre.text)
            
            instructors_temp = driver.find_elements_by_xpath("/html/body/div[1]/div/div/div[2]/section[5]/section/div/div/div/h3")

            instructors = []
            for name in instructors_temp:
                instructors.append(name.text)
                
            price =  0
        
        return syllabus, ratings_count, instructors, enrolled_students_count, avg_rating, num_reviews, price, ret_prerequisites
    
        
    scraped_items = scraping_udacity_course(url, prerequisites)

    # Mapped all data to dictionary
    # title
    course_dict[list_of_required_keys[0]] = title
    # url
    course_dict[list_of_required_keys[1]] = url
    # description
    course_dict[list_of_required_keys[2]] = description
    # syllabus
    course_dict[list_of_required_keys[3]] = scraped_items[0]
    # skills 
    course_dict[list_of_required_keys[4]] = skill
    # ratings_count
    course_dict[list_of_required_keys[5]] = scraped_items[1]
    # prerequisites
    course_dict[list_of_required_keys[6]] = scraped_items[7]
    # duration
    course_dict[list_of_required_keys[7]] = duration 
    # category 
    course_dict[list_of_required_keys[8]] = category
    # level
    course_dict[list_of_required_keys[9]] = level
    # schoolName
    course_dict[list_of_required_keys[10]] = 'Udacity Schools'
    # instructors
    course_dict[list_of_required_keys[11]] = scraped_items[2]
    # enrolled_students_count
    course_dict[list_of_required_keys[12]] = scraped_items[3]
    # avg_rating
    course_dict[list_of_required_keys[13]] = scraped_items[4]
    # num_reviews
    course_dict[list_of_required_keys[14]] = scraped_items[5]
    # price
    course_dict[list_of_required_keys[15]] = scraped_items[6]
    # source
    course_dict[list_of_required_keys[16]] = 'Udacity'

    return course_dict

In [11]:
course_dict = {}
list_of_udacity_courses = []

### Loop over courses url's to scrape data 

In [12]:
for i in range(len(courses_title)):
    list_of_udacity_courses.append(map_new_keys_for_udacity(courses_title[i], courses_urls[i], courses_duration[i],\
                                                            description[i], skills[i], prerequisites[i], levels[i],\
                                                            categories[i]))

In [13]:
list_of_udacity_courses[2]

{'title': 'Product Manager',
 'url': 'https://www.udacity.com/course/product-manager-nanodegree--nd036',
 'description': 'Envision and execute the development of industry-defining products, and learn how to successfully bring them to market.',
 'syllabus': 'Not Available',
 'skills': 'Product Strategy, Product Design, Product Development, Design Sprint, Product Launch',
 'ratings_count': '700',
 'prerequisites': 'No Experience Required',
 'duration': '4 Months',
 'category': 'nanodegree',
 'level': 'beginner',
 'schoolName': 'Udacity Schools',
 'instructors': ['Anastasia Root', 'Alex King', 'Yuva Murugan'],
 'enrolled_students_count': 'Not Available',
 'avg_rating': '4.7',
 'num_reviews': '700',
 'price': 'EGP 22076',
 'source': 'Udacity'}

### Convert list of udacity courses to json object

In [14]:
udacity_courses_json = json.dumps(list_of_udacity_courses, indent = 4)

### Udemy courses scraping

In [30]:
# Choise small number of courses scraped from udemy to reduce run time
NO_OF_UDEMY_COURSES = 10

In [31]:
udemy = Udemy(CLIENT_ID, CLIENT_SECRET)

In [32]:
# select the category of courses
courses_beginner = udemy.courses(page_size = NO_OF_UDEMY_COURSES, instructional_level = 'beginner')
courses_intermediate = udemy.courses(page_size = NO_OF_UDEMY_COURSES, instructional_level = 'intermediate')
courses_excpert = udemy.courses(page_size = NO_OF_UDEMY_COURSES, instructional_level = 'expert')

In [33]:
# keys of courses_beginner to select the key of data
print(courses_beginner.keys())

dict_keys(['count', 'next', 'previous', 'results', 'aggregations', 'search_tracking_id'])


In [34]:
# keys of data of each course 
courses_beginner['results'][0].keys()

dict_keys(['_class', 'id', 'title', 'url', 'is_paid', 'price', 'price_detail', 'price_serve_tracking_id', 'visible_instructors', 'image_125_H', 'image_240x135', 'is_practice_test_course', 'image_480x270', 'published_title', 'tracking_id', 'predictive_score', 'relevancy_score', 'input_features', 'lecture_search_result', 'curriculum_lectures', 'order_in_results', 'curriculum_items', 'headline', 'instructor_name'])

In [35]:
course_dict = {}
list_of_udemy_courses = []

### Create List of Dictionay of courses scraped from Udemy

In [36]:
def map_new_keys_for_udemy(course, level, list_of_keys = list_of_required_keys):
    course_dict = {}
    
    def scraping_udemy_course(course_url):
        # Get html code from source page 
        driver.get(course_url)
        time.sleep(5)
        
        avg_rating_temp = driver.find_elements_by_xpath("/html/body/div[1]/div[3]/div[2]/div[3]/div/div/div[4]/div/div[1]/div[2]/div/div[1]/div/div/a/span[1]/span[2]")
        avg_rating = avg_rating_temp[0].text
        
        enrolled_students_count_temp = driver.find_elements_by_xpath("/html/body/div[1]/div[3]/div[2]/div[3]/div/div/div[4]/div/div[1]/div[2]/div/div[2]/div")
        enrolled_students_count = re.split(' s', (enrolled_students_count_temp[0].text))[0]
        
        syllabus_temp = driver.find_elements_by_xpath("/html/body/div[1]/div[3]/div[2]/div[4]/div[2]/div/div/div/div/div/ul/li/div")
        
        syllabus = []
        for s in syllabus_temp:
            syllabus.append(s.text)
        
        
        ratings_count_temp = driver.find_elements_by_xpath("/html/body/div[1]/div[3]/div[2]/div[3]/div/div/div[4]/div/div[1]/div[2]/div/div[1]/div/div/a/span[2]")
        ratings_count = re.split(' r', (ratings_count_temp[0].text))[0][1:]
        
        
        duration_temp = driver.find_elements_by_xpath("/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div/div[1]/div/div[1]/div[2]/div/div[4]/div/ul/li[1]/div/div/span")
        duration = re.split(' o', (duration_temp[0].text))[0]
        
        instructors_temp = driver.find_elements_by_xpath("/html/body/div[1]/div[3]/div[2]/div[3]/div/div/div[4]/div/div[1]/div[3]/div/div/div/div/span/a/span")
        
        instructors = []
        for i in instructors_temp:
            instructors.append(i.text)
            
            
        prerequisites_temp = driver.find_elements_by_xpath("/html/body/div[1]/div[3]/div[2]/div[4]/div[6]/div/div/div/div/ul/li/div/div")
        
        prerequisites = []
        for p in prerequisites_temp:
            prerequisites.append(p.text)
        
        return syllabus, ratings_count, prerequisites, duration, instructors, enrolled_students_count, avg_rating
        
        
    scraped_items = scraping_udemy_course(UDEMY_URL + course['url'])


    # title
    course_dict[list_of_required_keys[0]] = course['title']
    # url
    course_dict[list_of_required_keys[1]] = UDEMY_URL + course['url']
    # description
    course_dict[list_of_required_keys[2]] = course['headline']
    # syllabus
    course_dict[list_of_required_keys[3]] = scraped_items[0]
    # skills 
    course_dict[list_of_required_keys[4]] = 'Not Available'
    # ratings_count
    course_dict[list_of_required_keys[5]] = scraped_items[1]
    # prerequisites
    course_dict[list_of_required_keys[6]] = scraped_items[2]
    # duration
    course_dict[list_of_required_keys[7]] = scraped_items[3]
    # category 
    course_dict[list_of_required_keys[8]] = course['_class']
    # level
    course_dict[list_of_required_keys[9]] = level
    # schoolName
    course_dict[list_of_required_keys[10]] = "Provided by Udemy Instructors"
    # instructors
    course_dict[list_of_required_keys[11]] = scraped_items[4]
    # enrolled_students_count
    course_dict[list_of_required_keys[12]] = scraped_items[5]
    # avg_rating
    course_dict[list_of_required_keys[13]] = scraped_items[6]
    # num_reviews
    course_dict[list_of_required_keys[14]] = udemy.course_reviews(course['id'])['count']
    # price
    course_dict[list_of_required_keys[15]] = course['price']
    # source
    course_dict[list_of_required_keys[16]] = 'Udemy'

    return course_dict

### Loop over courses url's to scrape data 

In [39]:
for course in courses_beginner['results']:
    list_of_udemy_courses.append(map_new_keys_for_udemy(course, 'beginner'))
    
for course in courses_intermediate['results']:
    list_of_udemy_courses.append(map_new_keys_for_udemy(course, 'intermediate'))
    
for course in courses_excpert['results']:
    list_of_udemy_courses.append(map_new_keys_for_udemy(course, 'excpert'))

In [40]:
list_of_udemy_courses[2]

{'title': 'Dynamic Cloth for Games in Maya for Unreal',
 'url': 'https://www.udemy.com/course/3dmotive-dynamic-cloth-for-games-with-maya-and-udk/',
 'description': 'Learn how to create dynamic cloth elements for real-time game environments!',
 'syllabus': ['Learn how to implement dynamic cloth into Maya and UDK',
  'Learn APEX clothing used in Maya and UDK'],
 'skills': 'Not Available',
 'ratings_count': '7',
 'prerequisites': ["Autodesk's Maya", 'UDK'],
 'duration': '1 hour',
 'category': 'course',
 'level': 'excpert',
 'schoolName': 'Provided by Udemy Instructors',
 'instructors': ['3dmotive LLC'],
 'enrolled_students_count': '150',
 'avg_rating': '3.7',
 'num_reviews': 7,
 'price': '$19.99',
 'source': 'Udemy'}

### Convert list of udemy courses to json object

In [41]:
udemy_courses_json = json.dumps(list_of_udemy_courses, indent = 4)

### store two courses list scraped from udacity and udemy in Mongo DB (Udemy_Udacity_Scraped_courses)

In [42]:
# Connect to my NOSQL DB (Mongo DB) Udemy_Udacity_Scraped_courses
# 27017 is the default port number for (Mongo DB)
client = MongoClient('localhost',27017)  
db = client.Udemy_Udacity_Scraped_courses

In [43]:
# Create Collection in DB
courses_collection = db.courses
#insert courses list into DB
courses_collection.insert_many(list_of_udemy_courses)
courses_collection.insert_many(list_of_udacity_courses)

### Get all courses after concatenation in database

In [44]:
all_courses = db["courses"]

In [45]:
Concat_courses = list(all_courses.find({}))

### Store List of Courses in DataFrame to work on data in seprated file

In [46]:
import pandas as pd

In [47]:
Courses_df = pd.DataFrame(Concat_courses)

In [48]:
Courses_df.to_csv("all_courses.csv")